In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm, expon, lognorm, uniform

import gpflow
from gpflowopt.domain import ContinuousParameter
from gpflowopt.bo import BayesianOptimizer
from gpflowopt.design import RandomDesign, LatinHyperCube
from gpflowopt.acquisition import ExpectedImprovement, ProbabilityOfImprovement
from gpflowopt.optim import SciPyOptimizer, StagedOptimizer, MCOptimizer

import data
import svm
import mlp
import custom_classes 

In [ ]:
def plot_gp(model, fn, r, domain=[-10,10]):
    x_new = np.linspace(domain[0], domain[1], 100).reshape(100, 1)
    mean, var = model.predict_y(x_new)
    plt.plot(r.x, fn(r.x), 'ro')
    plt.plot(x_new, fn(x_new), 'r')
    line, = plt.plot(x_new, mean, 'C0', lw=2)
    _ = plt.fill_between(x_new[:,0], mean[:,0] - 2*np.sqrt(var[:,0]), mean[:,0] + 2*np.sqrt(var[:,0]), color=line.get_color(), alpha=0.2)

# Bias effect

In [ ]:
fn1 = data.function1(x0=[1, 5, 6, 11], scale=1)
domain = ContinuousParameter('x', 0, 10) 
rd = RandomDesign(10, domain)
X = rd.generate()

In [ ]:
def test_bias():
    Y = fn1(X)
    model = gpflow.gpr.GPR(X, Y, gpflow.kernels.Matern52(1))

    alpha = ExpectedImprovement(model)

    acquisition_opt =  SciPyOptimizer(domain)
    optimizer = BayesianOptimizer(domain, alpha, optimizer=acquisition_opt, verbose=True)

    
    r = optimizer.optimize(fn1, n_iter=20)
    plot_gp(optimizer.acquisition.models[0],fn1, r, domain=[0, 10])
    plt.show()
    return(r)

In [ ]:
r = test_bias()

# Curse of dimensionality

In [ ]:
def test_dimension(dim=2, x0=0.0, b=0.0, init=10):
    domain = custom_classes.MyCube([-10,10], dim)
    rd = RandomDesign(init, domain)
    X = rd.generate()
    fn = lambda x:np.reshape(np.sum((x-x0)**2, axis=1) + b, [x.shape[0], 1])
    Y = fn(X)
    
    model = gpflow.gpr.GPR(X, Y, gpflow.kernels.Matern12(dim))
    alpha = ExpectedImprovement(model)
    acquisition_opt =  StagedOptimizer([MCOptimizer(domain, 200), SciPyOptimizer(domain)])
    optimizer = BayesianOptimizer(domain, alpha, optimizer=acquisition_opt, verbose=True)

    r = optimizer.optimize(fn, n_iter=10)
    print(r)
    return r

In [ ]:
r=test_dimension(3, 5, 0, 50)

# Uncertainty of measurement

In [ ]:
def test_uncertainty(noise=0.001, init=10):
    domain = custom_classes.MyCube([-10,10], 1)
    rd = RandomDesign(init, domain)
    X = rd.generate()
    fn = lambda x:data.function2(scale=-1e-3)(x) + np.random.randn(x.shape[1], 1)*noise
    Y = fn(X)
    
    model = gpflow.gpr.GPR(X, Y, gpflow.kernels.Matern52(1))
    alpha = ExpectedImprovement(model)
    acquisition_opt =  StagedOptimizer([MCOptimizer(domain, 200), SciPyOptimizer(domain)])
    optimizer = BayesianOptimizer(domain, alpha, optimizer=acquisition_opt, verbose=True)

    r = optimizer.optimize(fn, n_iter=10)
    plot_gp(optimizer.acquisition.models[0],fn, r)
    plt.show()
    return r

In [ ]:
test_uncertainty()

# Peaked vs wiggly

In [ ]:
x = np.linspace(-10,10,1000)
y = data.function3(w=3, k=0)(x)
plt.plot(x,y)

In [ ]:
def test_peak(w=3, k=0, init=10):
    domain = custom_classes.MyCube([-10,10], 1)
    rd = RandomDesign(init, domain)
    X = rd.generate()
    fn = data.function3(w=w, k=k)
    Y = fn(X)
    
    model = gpflow.gpr.GPR(X, Y, gpflow.kernels.Matern12(1))
    
    alpha = ExpectedImprovement(model)
    acquisition_opt =  StagedOptimizer([MCOptimizer(domain, 200), SciPyOptimizer(domain)])
    optimizer = BayesianOptimizer(domain, alpha, optimizer=acquisition_opt, verbose=True)

    r = optimizer.optimize(fn, n_iter=100)
    plot_gp(optimizer.acquisition.models[0],fn, r)
    plt.show()
    return r

In [ ]:
test_peak()

# Model selection: SVM

## Grid search

In [ ]:
grid = dict(C=np.logspace(-6, -1, 10), gamma=np.logspace(-3, -1, 4))
gs_model, test_acc = svm.grid_search(grid)
print("Best C = %e, Best gamma = %e" % (gs_model.best_estimator_.C, gs_model.best_estimator_.gamma))
print("Test acc = %.3f" % (test_acc))

## Random search

In [ ]:
grid = dict(C=uniform(1e-6, 1e-1), gamma=uniform(1e-6, 1e-1))
gs_model, test_acc = svm.random_search(grid)
print("Best C = %e, Best gamma = %e" % (gs_model.best_estimator_.C, gs_model.best_estimator_.gamma))
print("Test acc = %.3f" % (test_acc))

## Bayesian opt

In [ ]:
train, val, test = svm.get_data(0.7, 0.1)

In [ ]:
# Write BO for model selection 

# Model selection: MLP

## Grid search

In [ ]:
grid = {'learning_rate':np.logspace(-3, -1, 3), 'batch_size':[50, 100, 200], 'n_hidden_1':[128, 256], 'n_hidden_2':[128, 256]}
best_p, best_err = mlp.grid_search(grid)

In [ ]:
_, test_acc = mlp.train_predict(best_p[0], best_p[1], best_p[2], best_p[3])

## Random search

In [ ]:
grid = {'learning_rate':[-4, -1], 'batch_size':[50, 200], 'n_hidden_1':[128, 256], 'n_hidden_2':[128, 256]}
best_p, best_err = mlp.random_search(grid, 30)

In [ ]:
_, test_acc = mlp.train_predict(best_p[0], best_p[1][0], best_p[2][0], best_p[3][0])

## Bayesian opt

In [ ]:
# Write BO for model selection 